In [ ]:
# Transform Categorical Data to Numerical using One-Hot Encoding

print("=" * 70)
print("ONE-HOT ENCODING CATEGORICAL VARIABLES")
print("=" * 70)

# Identify categorical columns (object type)
all_categorical_cols = data.select_dtypes(include='object').columns.tolist()

# Exclude columns that shouldn't be one-hot encoded
exclude_from_encoding = [
    'Accident_Index',           # Unique identifier
    'LSOA_of_Accident_Location', # High cardinality location code
    'Location_Easting_OSGR',    # Coordinate data
    'Location_Northing_OSGR'    # Coordinate data
]

# Filter to only columns we want to encode
categorical_cols = [col for col in all_categorical_cols if col not in exclude_from_encoding]

print(f"\nOriginal dataset shape: {data.shape}")
print(f"\nCategorical columns to encode ({len(categorical_cols)}):")
for col in categorical_cols:
    n_unique = data[col].nunique()
    print(f"  • {col:35s} - {n_unique} unique values")

print(f"\nColumns excluded from encoding ({len(exclude_from_encoding)}):")
for col in exclude_from_encoding:
    if col in all_categorical_cols:
        print(f"  • {col}")

# Apply one-hot encoding only to selected columns
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=False, dtype=int)

print("\n" + "-" * 70)
print("ENCODING COMPLETE")
print("-" * 70)
print(f"New dataset shape: {data_encoded.shape}")
print(f"Columns added: {data_encoded.shape[1] - data.shape[1]}")
print(f"Categorical columns encoded: {len(categorical_cols)}")

# Update data variable
data = data_encoded

# Display sample of new columns
print("\n" + "-" * 70)
print("SAMPLE OF ENCODED COLUMNS (first 20):")
print("-" * 70)
encoded_cols = [col for col in data.columns if any(cat in col for cat in categorical_cols)]
for col in encoded_cols[:20]:
    print(f"  • {col}")
if len(encoded_cols) > 20:
    print(f"  ... and {len(encoded_cols) - 20} more encoded columns")

print("\n" + "=" * 70)
print(f"Data is now ready for modeling!")
print(f"Final shape: {data.shape}")
print("=" * 70)

In [ ]:
# K-Means Clustering for Optimal Hospital Placement
# With weighting based on Speed Limit and Road Density

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance_matrix
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION: Set number of clusters and weighting parameters
# ============================================================================
K = 4  # <- CHANGE THIS VALUE to adjust number of hospital locations

# Weighting configuration
SPEED_WEIGHT_ENABLED = True      # Enable/disable speed limit weighting
DENSITY_WEIGHT_ENABLED = True    # Enable/disable road density weighting
DENSITY_RADIUS_KM = 2.5          # Radius (in km) for calculating local accident density

print("=" * 80)
print(f"K-MEANS CLUSTERING FOR OPTIMAL HOSPITAL PLACEMENT (K={K})")
print(f"Speed weighting: {'ON' if SPEED_WEIGHT_ENABLED else 'OFF'}")
print(f"Density weighting: {'ON' if DENSITY_WEIGHT_ENABLED else 'OFF'} (radius={DENSITY_RADIUS_KM}km)")
print("=" * 80)

# ============================================================================
# STEP 1: Prepare Spatial Data with Weighting
# ============================================================================
print("\n[STEP 1] Preparing Spatial Data with Speed Limit and Density Weighting")
print("-" * 80)

# Recreate Speed_Limit_Grouped from one-hot encoded columns
data_cluster = data.copy()

# Reconstruct Speed_Limit_Grouped
speed_cols = [col for col in data.columns if 'Speed_Limit_Grouped_' in col]
for col in speed_cols:
    speed_group = col.replace('Speed_Limit_Grouped_', '')
    data_cluster.loc[data[col] == 1, 'Speed_Limit_Grouped'] = speed_group

# Create clustering dataset with required columns
clustering_data = data_cluster[['Longitude', 'Latitude', 'Speed_Limit_Grouped']].copy()

# Initialize combined weight as 1.0 (no weighting)
clustering_data['Combined_Weight'] = 1.0

# ============================================================================
# Speed Limit Weighting
# ============================================================================
if SPEED_WEIGHT_ENABLED:
    speed_weights = {
        '30_or_less': 1.0,    # Baseline (no extra weight)
        '40-50': 1.1,          # Slight increase (10% more weight)
        '60_plus': 1.2         # Modest increase (20% more weight)
    }
    clustering_data['Speed_Weight'] = clustering_data['Speed_Limit_Grouped'].map(speed_weights).fillna(1.0)
    clustering_data['Combined_Weight'] *= clustering_data['Speed_Weight']
    print("Speed limit weighting applied")
else:
    clustering_data['Speed_Weight'] = 1.0

# ============================================================================
# Road Density Weighting (Local Accident Density)
# ============================================================================
if DENSITY_WEIGHT_ENABLED:
    print(f"\nCalculating local accident density (radius={DENSITY_RADIUS_KM}km)...")
    
    # Convert lat/lon to approximate distances (1 degree ≈ 111km at equator)
    # For small areas, this approximation is sufficient
    coords = clustering_data[['Longitude', 'Latitude']].values
    
    # Calculate pairwise distances (in degrees)
    distances = distance_matrix(coords, coords)
    
    # Convert radius from km to degrees (approximate)
    radius_degrees = DENSITY_RADIUS_KM / 111.0
    
    # Count neighbors within radius for each point
    neighbors_within_radius = (distances <= radius_degrees).sum(axis=1) - 1  # Subtract 1 to exclude self
    
    # Normalize density to create weights (higher density = higher weight)
    # Min-max normalization to range [1.0, 1.5]
    min_neighbors = neighbors_within_radius.min()
    max_neighbors = neighbors_within_radius.max()
    
    if max_neighbors > min_neighbors:
        density_weights = 1.0 + 0.5 * (neighbors_within_radius - min_neighbors) / (max_neighbors - min_neighbors)
    else:
        density_weights = np.ones(len(neighbors_within_radius))
    
    clustering_data['Density_Weight'] = density_weights
    clustering_data['Neighbors_Count'] = neighbors_within_radius
    clustering_data['Combined_Weight'] *= clustering_data['Density_Weight']
    
    print(f"  Min neighbors within {DENSITY_RADIUS_KM}km: {min_neighbors}")
    print(f"  Max neighbors within {DENSITY_RADIUS_KM}km: {max_neighbors}")
    print(f"  Mean neighbors: {neighbors_within_radius.mean():.1f}")
    print("Road density weighting applied")
else:
    clustering_data['Density_Weight'] = 1.0
    clustering_data['Neighbors_Count'] = 0

# ============================================================================
# Summary Statistics
# ============================================================================
print(f"\nTotal accidents to cluster: {len(clustering_data)}")
print(f"\nSpeed limit distribution:")
print(clustering_data['Speed_Limit_Grouped'].value_counts())
print(f"\nCombined weight range: {clustering_data['Combined_Weight'].min():.2f} - {clustering_data['Combined_Weight'].max():.2f}")
print(f"Mean combined weight: {clustering_data['Combined_Weight'].mean():.2f}")

# ============================================================================
# STEP 2: Apply K-Means with Sample Weighting
# ============================================================================
print(f"\n[STEP 2] Applying K-Means Clustering (k={K})")
print("-" * 80)

# Use UNWEIGHTED coordinates for clustering, but use sample_weight parameter
X = clustering_data[['Longitude', 'Latitude']].values

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply K-Means with k=K and combined weights
kmeans = KMeans(n_clusters=K, random_state=42, n_init=10)
clustering_data['Cluster'] = kmeans.fit_predict(X_scaled, sample_weight=clustering_data['Combined_Weight'].values)

# Get cluster centers in original coordinate space
cluster_centers_scaled = kmeans.cluster_centers_
optimal_hospital_locations = scaler.inverse_transform(cluster_centers_scaled)

print("Clustering complete!")
print(f"Number of clusters: {K}")

# ============================================================================
# STEP 3: Cluster Analysis
# ============================================================================
print("\n[STEP 3] Cluster Analysis")
print("-" * 80)

for i in range(K):
    cluster_data = clustering_data[clustering_data['Cluster'] == i]
    n_accidents = len(cluster_data)
    pct = (n_accidents / len(clustering_data)) * 100
    
    # Speed distribution in cluster
    speed_dist = cluster_data['Speed_Limit_Grouped'].value_counts()
    
    print(f"\nCluster {i}:")
    print(f"  Number of accidents: {n_accidents} ({pct:.1f}%)")
    print(f"  Optimal Hospital Location (Centroid):")
    print(f"    Longitude: {optimal_hospital_locations[i][0]:.6f}")
    print(f"    Latitude:  {optimal_hospital_locations[i][1]:.6f}")
    print(f"  Average combined weight: {cluster_data['Combined_Weight'].mean():.2f}")
    
    if SPEED_WEIGHT_ENABLED:
        print(f"  Average speed weight: {cluster_data['Speed_Weight'].mean():.2f}")
    
    if DENSITY_WEIGHT_ENABLED:
        print(f"  Average density weight: {cluster_data['Density_Weight'].mean():.2f}")
        print(f"  Average neighbors within {DENSITY_RADIUS_KM}km: {cluster_data['Neighbors_Count'].mean():.1f}")
    
    print(f"  Speed limit distribution:")
    for speed, count in speed_dist.items():
        print(f"    {speed}: {count} ({count/n_accidents*100:.1f}%)")
    print(f"  Longitude range: {cluster_data['Longitude'].min():.6f} to {cluster_data['Longitude'].max():.6f}")
    print(f"  Latitude range:  {cluster_data['Latitude'].min():.6f} to {cluster_data['Latitude'].max():.6f}")

# ============================================================================
# STEP 4: Visualization
# ============================================================================
print("\n[STEP 4] Visualizing Results")
print("-" * 80)

fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Define colors (cycle through if K > 10)
color_palette = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
colors = color_palette * ((K // len(color_palette)) + 1)

# ============================================================================
# Plot 1: Cluster Assignment
# ============================================================================
for i in range(K):
    cluster_points = clustering_data[clustering_data['Cluster'] == i]
    axes[0].scatter(cluster_points['Longitude'], cluster_points['Latitude'], 
                    c=colors[i], alpha=0.5, s=30, label=f'Cluster {i} ({len(cluster_points)} accidents)')

# Plot optimal hospital locations (centroids)
axes[0].scatter(optimal_hospital_locations[:, 0], optimal_hospital_locations[:, 1], 
                c='black', marker='*', s=500, edgecolors='yellow', linewidths=2,
                label='Optimal Hospital Locations', zorder=5)

# Annotate hospital locations
for i in range(K):
    axes[0].annotate(f'Hospital {i}', 
                     xy=(optimal_hospital_locations[i][0], optimal_hospital_locations[i][1]),
                     xytext=(10, 10), textcoords='offset points',
                     fontsize=12, fontweight='bold',
                     bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7))

axes[0].set_xlabel('Longitude', fontsize=12)
axes[0].set_ylabel('Latitude', fontsize=12)
axes[0].set_title(f'Cluster Assignment (k={K})', fontsize=14, fontweight='bold')
axes[0].legend(loc='best', fontsize=9)
axes[0].grid(True, alpha=0.3)

# ============================================================================
# Plot 2: Density Heatmap (if density weighting enabled)
# ============================================================================
if DENSITY_WEIGHT_ENABLED:
    scatter = axes[1].scatter(clustering_data['Longitude'], clustering_data['Latitude'], 
                              c=clustering_data['Density_Weight'], cmap='YlOrRd', 
                              alpha=0.6, s=40, edgecolors='black', linewidths=0.3)
    
    # Plot hospital locations on density map
    axes[1].scatter(optimal_hospital_locations[:, 0], optimal_hospital_locations[:, 1], 
                    c='blue', marker='*', s=500, edgecolors='white', linewidths=2,
                    label='Hospital Locations', zorder=5)
    
    plt.colorbar(scatter, ax=axes[1], label='Density Weight')
    axes[1].set_xlabel('Longitude', fontsize=12)
    axes[1].set_ylabel('Latitude', fontsize=12)
    axes[1].set_title(f'Accident Density Heatmap (radius={DENSITY_RADIUS_KM}km)', fontsize=14, fontweight='bold')
    axes[1].legend(loc='best', fontsize=10)
    axes[1].grid(True, alpha=0.3)
else:
    # Just show speed weights if density disabled
    scatter = axes[1].scatter(clustering_data['Longitude'], clustering_data['Latitude'], 
                              c=clustering_data['Speed_Weight'], cmap='coolwarm', 
                              alpha=0.6, s=40, edgecolors='black', linewidths=0.3)
    
    axes[1].scatter(optimal_hospital_locations[:, 0], optimal_hospital_locations[:, 1], 
                    c='black', marker='*', s=500, edgecolors='yellow', linewidths=2,
                    label='Hospital Locations', zorder=5)
    
    plt.colorbar(scatter, ax=axes[1], label='Speed Weight')
    axes[1].set_xlabel('Longitude', fontsize=12)
    axes[1].set_ylabel('Latitude', fontsize=12)
    axes[1].set_title('Speed Limit Weight Distribution', fontsize=14, fontweight='bold')
    axes[1].legend(loc='best', fontsize=10)
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================================================
# STEP 5: Recommendations
# ============================================================================
print("\n[STEP 5] Hospital Placement Recommendations")
print("=" * 80)

for i in range(K):
    cluster_data = clustering_data[clustering_data['Cluster'] == i]
    high_speed_count = len(cluster_data[cluster_data['Speed_Limit_Grouped'] == '60_plus'])
    high_speed_pct = (high_speed_count / len(cluster_data) * 100) if len(cluster_data) > 0 else 0
    
    print(f"\nHospital {i} Location:")
    print(f"  Coordinates: ({optimal_hospital_locations[i][1]:.6f}, {optimal_hospital_locations[i][0]:.6f})")
    print(f"  Will serve: {len(cluster_data)} accidents")
    print(f"  High-speed road accidents (60+): {high_speed_count} ({high_speed_pct:.1f}%)")
    
    if DENSITY_WEIGHT_ENABLED:
        print(f"  Average local density: {cluster_data['Neighbors_Count'].mean():.1f} accidents within {DENSITY_RADIUS_KM}km")

print("\n" + "=" * 80)
print(f"Clustering complete! {K} hospital locations optimized.")
print("=" * 80)

# Add cluster labels back to main dataset (matching by index)
data = data.reset_index(drop=True)
clustering_data = clustering_data.reset_index(drop=True)
data['Hospital_Cluster'] = clustering_data['Cluster']